In [1]:
import os
import pandas as pd
import glob
import menpo.io as mio
import numpy as np
#np.random.seed(1337)  # for reproducibility
from keras.utils import np_utils
from keras import backend as K

Using TensorFlow backend.


In [2]:
# returns (X_train, y_train), (X_test, y_test)
def randomSplit(X, y, propTest):
    assert X.shape[0] == y.shape[0]
    p = np.random.permutation(len(y))
    sX, sy = X[p], y[p]
    cutoff = round(propTest * len(y))
    return (sX[cutoff:], sy[cutoff:]), (sX[0:cutoff], sy[0:cutoff])

def randomOrder(X, y):
    p = np.random.permutation(len(y))
    return X[p], y[p]

# based on
# http://www.socouldanyone.com/2013/03/converting-grayscale-to-rgb-with-numpy.html
def to_rgb(im):
    if im.n_channels == 1:
        return np.vstack([im.pixels.astype(np.uint8)] * 3)
    else:
        return im.pixels

def subsample(df, col, props = None):
    labels = df[col].get_values()
    counts = dict()
    for l in labels:
        counts[l] = counts.get(l, 0) + 1
    countsOfProps = dict()
    for l in set(labels):
        countsOfProps[l] = counts.get(l, 0) * props.get(l, 0)
    leastKey = min(countsOfProps, key=countsOfProps.get)
    leastVal = round(countsOfProps[leastKey])
    thresholds = dict()
    for l in set(labels):
        if props == None or not leastKey in props: # assume equal parts
            thresholds[l] = leastVal
        else:
            labelToLabel = props.get(l, 0) / props[leastKey]
            thresholds[l] = round(labelToLabel * leastVal)
    df = df.reindex(np.random.permutation(df.index))
    enough = dict()
    ret = pd.DataFrame(index=np.arange(0, leastVal / props[leastKey]), columns=df.columns)
    i = 0
    for index, r in df.iterrows():
        if(enough.get(r[col], 0) <= thresholds[r[col]]):
            ret.loc[i] = r
            enough[r[col]] = enough.get(r[col], 0) + 1
            i += 1
    return ret


In [3]:
data_file = 'gold.tsv'
image_folder = './cropped/' # must end with '/'
variable = 'ow' # relevant column of data_file
height = 128
width = 128
channels = 3

In [4]:
annos = pd.DataFrame.from_csv(data_file, header=0, index_col=None, sep="\t")
props = { 1: 0.5, 0: 0.5 }
annos = subsample(annos, variable, props)

ids = list(set(annos['subject_id'].get_values()))
ids = np.random.permutation(ids)
boundary = round(0.8 * len(ids))
train_ids = set(ids[0:boundary])
test_ids = set(ids[boundary:])

train_images = []
train_labels = []
test_images = []
test_labels = []
for fp in glob.glob(image_folder + '*.jpg'):
    fn = os.path.basename(fp)
    image = mio.import_image(fp)
    assert(image.shape == (height, width))
    assert(image.n_channels == channels)
    # if bool(np.random.randint(0,1)):
    #     image = image.mirror()
    id = annos.loc[annos['file_name'] == fn]['subject_id'].sum()
    if id in train_ids:
        train_images.append(image)
        train_labels.append(annos.loc[annos['file_name'] == fn][variable].sum())
    elif id in test_ids:
        test_images.append(image)
        test_labels.append(annos.loc[annos['file_name'] == fn][variable].sum())


In [5]:
nb_classes = 2

# input image dimensions
img_rows, img_cols, img_channels = height, width, channels

X_train = np.stack([img.pixels for img in train_images])
X_test = np.stack([img.pixels for img in test_images])
if K.image_dim_ordering() == 'tf':
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, channels)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, channels)
    input_shape = input_shape = (img_rows, img_cols, channels)
else:
    input_shape = (channels, img_rows, img_cols)
    
y_train = np.array(train_labels)
y_test = np.array(test_labels)
#(X_train, y_train), (X_test, y_test) = randomSplit(Xs, ys, 0.1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to trinary class matrices
y_train = np_utils.to_categorical(y_train, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)

print('y_train shape:', y_train.shape)

X_train, y_train = randomOrder(X_train, y_train)
X_test, y_test = randomOrder(X_test, y_test)

X_train shape: (2635, 128, 128, 3)
2635 train samples
679 test samples
y_train shape: (2635, 2)


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D

batch_size = 50
nb_epoch = 2

# number of convolutional filters to use
nb_filters1 = 48
# convolution kernel size
kernel1 = (7, 7)
# size of pooling area for max pooling
pool1 = (3, 3)
# strides for pooling
strides1 = (2, 2)

# number of convolutional filters to use
nb_filters2 = 128
# convolution kernel size
kernel2 = (5, 5)
# size of pooling area for max pooling
pool2 = (3, 3)
# strides for pooling
strides2 = (2, 2)

# number of convolutional filters to use
nb_filters3 = 192
# convolution kernel size
kernel3 = (3, 3)
# size of pooling area for max pooling
pool3 = (3, 3)
# strides for pooling
strides3 = (2, 2)

dense1 = 256
dense2 = 256

# keras.applications.vgg16.VGG16(include_top=True, weights='imagenet', input_tensor=None, input_shape=None)

model = Sequential()

model.add(Convolution2D(nb_filters1, kernel1[0], kernel1[1],
                        border_mode='same',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool1, strides=strides1))
#model.add(Dropout(0.2))
model.add(Convolution2D(nb_filters2, kernel2[0], kernel2[1], border_mode='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool2, strides=strides2))
#model.add(Dropout(0.2))
model.add(Convolution2D(nb_filters3, kernel3[0], kernel3[1], border_mode='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool3, strides=strides3))
#model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(dense1))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(dense2))
model.add(Activation('relu'))
model.add(Dropout(0.5))
# model.add(Dense(1))
# model.add(Activation('sigmoid'))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True)

#datagen.fit(X_train)

In [ ]:
for l in model.layers:
    print(l.name + ": " + str(l.input_shape) + "->" + str(l.output_shape))

In [ ]:
# fits the model on batches with real-time data augmentation:
model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size),
                    samples_per_epoch=len(X_train), nb_epoch=nb_epoch)

score = model.evaluate(X_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Epoch 1/2
1000/2635 [==========>...................] - ETA: 170s - loss: 0.9993 - acc: 0.4920

In [ ]:
model.save("overweight_classifier.h5")